In [1]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn.functional as F 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
train = pd.read_csv("C:/Users/moham/OneDrive/Documents/Desktop/JIT/letter_train.csv")
train.replace(r'^\s*$', np.nan, regex=True, inplace=True)
train.dropna(inplace=True)


test = pd.read_csv("C:/Users/moham/OneDrive/Documents/Desktop/JIT/letter_test.csv")
test.replace(r'^\s*$', np.nan, regex=True, inplace=True)
test.dropna(inplace=True)
print(len(test.columns))

64


In [3]:
train

,gesture,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,z11,z12,z13,z14,z15,z16,z17,z18,z19,z20
0,A,0.752823,0.373358,-4.351319e-07,0.840682,0.315064,-0.017487,0.895263,0.235904,-0.020836,...,-0.012628,0.693979,0.165705,-0.048182,0.710418,0.220471,-0.042087,0.717044,0.269370,-0.026001
1,A,0.749799,0.389664,-5.321739e-07,0.842005,0.328627,-0.012226,0.894808,0.252766,-0.013331,...,-0.009322,0.695285,0.177125,-0.038746,0.711941,0.230007,-0.030610,0.718896,0.279309,-0.014276
2,A,0.754766,0.401334,-4.530839e-07,0.844320,0.345357,-0.017787,0.901676,0.266540,-0.020207,...,-0.007375,0.703929,0.188687,-0.039472,0.718742,0.245413,-0.032439,0.726482,0.295354,-0.016118
3,A,0.769240,0.422996,-4.014602e-07,0.856139,0.366092,-0.018306,0.910973,0.281576,-0.021523,...,-0.011983,0.705819,0.201553,-0.043019,0.723938,0.256046,-0.035374,0.732768,0.308078,-0.018747
4,A,0.775203,0.439014,-3.919496e-07,0.867564,0.380060,-0.011609,0.921898,0.300867,-0.011069,...,-0.008911,0.726265,0.218610,-0.036537,0.740347,0.270459,-0.027987,0.743678,0.319981,-0.011353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11469,Z,0.594302,0.906174,1.606369e-07,0.720756,0.894178,-0.034722,0.835709,0.834138,-0.050246,...,-0.024335,0.705019,0.608442,-0.086188,0.708979,0.688675,-0.084583,0.689652,0.751841,-0.067031
11470,Z,0.592576,0.901998,1.278450e-07,0.722727,0.890245,-0.032892,0.840114,0.829402,-0.047667,...,-0.025410,0.709261,0.602365,-0.088362,0.713679,0.682160,-0.087624,0.691764,0.747748,-0.070890
11471,Z,0.587568,0.892351,1.249735e-07,0.717597,0.887137,-0.034021,0.836185,0.827295,-0.047644,...,-0.027608,0.709409,0.592305,-0.094798,0.713589,0.674735,-0.096545,0.693192,0.744609,-0.080709
11472,Z,0.586861,0.894335,1.218530e-07,0.720712,0.885324,-0.030634,0.836220,0.824128,-0.044090,...,-0.031689,0.703991,0.589883,-0.092560,0.706967,0.669496,-0.090496,0.685686,0.733243,-0.073479


In [4]:
print(len(train.columns))

64


In [5]:
device = (
    "cpu"
)
print (device)
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers=nn.Sequential(
            nn.Linear(63,1024),
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512,364),
            nn.ReLU(),
            nn.Linear(364,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(64,29),
            # nn.ReLU(),
            # nn.Linear(32,29)
        )
    def forward(self,x):
        logits=self.layers(x)

        return logits

cpu


In [6]:
model=NeuralNetwork()

In [7]:
import torch.optim as optim
valid_loader= torch.utils.data.DataLoader(train, batch_size=4, shuffle=True)
classes=("A", "B", "C", "D", "del", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "nothing", "O", "P", "Q", "R", "S", "space", "T", "U", "V", "W", "X", "Y", "Z")

In [8]:
loss_fn=torch.nn.CrossEntropyLoss()
opt=optim.Adam(model.parameters(), lr=1e-3)

In [28]:
from torch.utils.data import Dataset

class GestureDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        # Map 'gesture' and 'Hand_Id' to numerical values
        self.label_mapping = {gesture: idx for idx, gesture in enumerate(self.data['gesture'].unique())}
        self.data['gesture'] = self.data['gesture'].map(self.label_mapping)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Include 'Hand_Id' as part of the input features
        features = self.data.iloc[idx, 1:].values.astype(float)  # All columns except 'gesture'
        features = torch.tensor(features, dtype=torch.float32)
        #features = torch.cat((torch.tensor([hand_id], dtype=torch.float32), features))  # Include 'Hand_Id'

        # Labels (gesture)
        label = torch.tensor(self.data.iloc[idx, 0], dtype=torch.long)  # 'gesture' column
                
        return features, label
tr=GestureDataset(train)
train_loder= torch.utils.data.DataLoader(tr, batch_size=40, shuffle=True)
test=GestureDataset(test)
test_loader=torch.utils.data.DataLoader(test)


ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [11]:
train

,gesture,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,z11,z12,z13,z14,z15,z16,z17,z18,z19,z20
0,0,0.752823,0.373358,-4.351319e-07,0.840682,0.315064,-0.017487,0.895263,0.235904,-0.020836,...,-0.012628,0.693979,0.165705,-0.048182,0.710418,0.220471,-0.042087,0.717044,0.269370,-0.026001
1,0,0.749799,0.389664,-5.321739e-07,0.842005,0.328627,-0.012226,0.894808,0.252766,-0.013331,...,-0.009322,0.695285,0.177125,-0.038746,0.711941,0.230007,-0.030610,0.718896,0.279309,-0.014276
2,0,0.754766,0.401334,-4.530839e-07,0.844320,0.345357,-0.017787,0.901676,0.266540,-0.020207,...,-0.007375,0.703929,0.188687,-0.039472,0.718742,0.245413,-0.032439,0.726482,0.295354,-0.016118
3,0,0.769240,0.422996,-4.014602e-07,0.856139,0.366092,-0.018306,0.910973,0.281576,-0.021523,...,-0.011983,0.705819,0.201553,-0.043019,0.723938,0.256046,-0.035374,0.732768,0.308078,-0.018747
4,0,0.775203,0.439014,-3.919496e-07,0.867564,0.380060,-0.011609,0.921898,0.300867,-0.011069,...,-0.008911,0.726265,0.218610,-0.036537,0.740347,0.270459,-0.027987,0.743678,0.319981,-0.011353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11469,27,0.594302,0.906174,1.606369e-07,0.720756,0.894178,-0.034722,0.835709,0.834138,-0.050246,...,-0.024335,0.705019,0.608442,-0.086188,0.708979,0.688675,-0.084583,0.689652,0.751841,-0.067031
11470,27,0.592576,0.901998,1.278450e-07,0.722727,0.890245,-0.032892,0.840114,0.829402,-0.047667,...,-0.025410,0.709261,0.602365,-0.088362,0.713679,0.682160,-0.087624,0.691764,0.747748,-0.070890
11471,27,0.587568,0.892351,1.249735e-07,0.717597,0.887137,-0.034021,0.836185,0.827295,-0.047644,...,-0.027608,0.709409,0.592305,-0.094798,0.713589,0.674735,-0.096545,0.693192,0.744609,-0.080709
11472,27,0.586861,0.894335,1.218530e-07,0.720712,0.885324,-0.030634,0.836220,0.824128,-0.044090,...,-0.031689,0.703991,0.589883,-0.092560,0.706967,0.669496,-0.090496,0.685686,0.733243,-0.073479


In [12]:
def acc(y_pred,y):
    _,pred=torch.max(y_pred,1)
    yes=(pred==y).sum().item()
    return yes/y.size(0)

In [15]:
from sklearn.preprocessing import StandardScaler
for epoch in range (70):
    ls=0
    model.train()
    for batch,(X,y) in enumerate(train_loder):
        X,y=X.to(device),y.to(device)
        mean=torch.nanmean(X)   
        X=torch.nan_to_num(X,nan=mean)
        #print(X)
        # print(f"Max value in X: {X.max()}")
        # print(f"Min value in X: {X.min()}")
        # print(f"Max value in y: {y.max()}")
        # print(f"Min value in y: {y.min()}")
       # X = (X - X.mean()) / X.std()

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        log=model(X)
        print("NaNs in model output:", torch.isnan(log).sum())
        #print(y)
        loss=loss_fn(log,y)
        print("NaNs in loss:", torch.isnan(loss).sum())
        opt.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        print("batch number",batch)
        print(acc(log,y))
        print("acc",loss.item())
        opt.step()
        ls+=loss.item()


NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 0
0.0
acc 3.3756725788116455
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 1
0.1
acc 3.3756611347198486
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 2
0.025
acc 3.3656907081604004
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 3
0.075
acc 3.339252471923828
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 4
0.0
acc 3.3886191844940186
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 5
0.05
acc 3.359619617462158
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 6
0.1
acc 3.389725923538208
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 7
0.05
acc 3.3489482402801514
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 8
0.0
acc 3.3581783771514893
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 9
0.025
acc 3.3770530223846436
NaNs in model o

In [18]:
train

,gesture,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,z11,z12,z13,z14,z15,z16,z17,z18,z19,z20
0,0,0.752823,0.373358,-4.351319e-07,0.840682,0.315064,-0.017487,0.895263,0.235904,-0.020836,...,-0.012628,0.693979,0.165705,-0.048182,0.710418,0.220471,-0.042087,0.717044,0.269370,-0.026001
1,0,0.749799,0.389664,-5.321739e-07,0.842005,0.328627,-0.012226,0.894808,0.252766,-0.013331,...,-0.009322,0.695285,0.177125,-0.038746,0.711941,0.230007,-0.030610,0.718896,0.279309,-0.014276
2,0,0.754766,0.401334,-4.530839e-07,0.844320,0.345357,-0.017787,0.901676,0.266540,-0.020207,...,-0.007375,0.703929,0.188687,-0.039472,0.718742,0.245413,-0.032439,0.726482,0.295354,-0.016118
3,0,0.769240,0.422996,-4.014602e-07,0.856139,0.366092,-0.018306,0.910973,0.281576,-0.021523,...,-0.011983,0.705819,0.201553,-0.043019,0.723938,0.256046,-0.035374,0.732768,0.308078,-0.018747
4,0,0.775203,0.439014,-3.919496e-07,0.867564,0.380060,-0.011609,0.921898,0.300867,-0.011069,...,-0.008911,0.726265,0.218610,-0.036537,0.740347,0.270459,-0.027987,0.743678,0.319981,-0.011353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11469,27,0.594302,0.906174,1.606369e-07,0.720756,0.894178,-0.034722,0.835709,0.834138,-0.050246,...,-0.024335,0.705019,0.608442,-0.086188,0.708979,0.688675,-0.084583,0.689652,0.751841,-0.067031
11470,27,0.592576,0.901998,1.278450e-07,0.722727,0.890245,-0.032892,0.840114,0.829402,-0.047667,...,-0.025410,0.709261,0.602365,-0.088362,0.713679,0.682160,-0.087624,0.691764,0.747748,-0.070890
11471,27,0.587568,0.892351,1.249735e-07,0.717597,0.887137,-0.034021,0.836185,0.827295,-0.047644,...,-0.027608,0.709409,0.592305,-0.094798,0.713589,0.674735,-0.096545,0.693192,0.744609,-0.080709
11472,27,0.586861,0.894335,1.218530e-07,0.720712,0.885324,-0.030634,0.836220,0.824128,-0.044090,...,-0.031689,0.703991,0.589883,-0.092560,0.706967,0.669496,-0.090496,0.685686,0.733243,-0.073479


In [16]:
model.eval()

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=63, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=364, bias=True)
    (5): ReLU()
    (6): Linear(in_features=364, out_features=128, bias=True)
    (7): ReLU()
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): Dropout(p=0.2, inplace=False)
    (10): ReLU()
    (11): Linear(in_features=64, out_features=29, bias=True)
  )
)

AttributeError: 'GestureDataset' object has no attribute 'columns'

In [27]:
sum=0
i=0
with torch.no_grad():
    for _,(x,y) in enumerate(test_loader):
        x,y=x.to(device),y.to(device)
        print(x)
        y_pred=model(x)
        accuracy=acc(y_pred,y)

        
        print("loss",loss_fn(y_pred,y))
        sum+=accuracy
        i+=1
        print(accuracy)
print("tot",(sum/i))

tensor([[ 4.7558e-01, -5.1378e-07,  3.5220e-01,  4.3363e-01, -3.6467e-02,
          4.3487e-01,  3.4907e-01, -4.2906e-02,  4.4416e-01,  2.5091e-01,
         -4.8296e-02,  4.3903e-01,  1.7416e-01, -5.0166e-02,  3.7295e-01,
          2.5813e-01,  1.0736e-03,  3.7311e-01,  1.8583e-01, -4.3392e-02,
          3.5624e-01,  2.4691e-01, -7.1120e-02,  3.5301e-01,  3.0849e-01,
         -8.0420e-02,  3.0464e-01,  2.5875e-01,  6.4313e-03,  3.0051e-01,
          1.9132e-01, -4.2774e-02,  2.9026e-01,  2.7216e-01, -5.8589e-02,
          2.9634e-01,  3.3649e-01, -5.4885e-02,  2.4224e-01,  2.7031e-01,
          3.0994e-03,  2.3663e-01,  2.0578e-01, -5.1192e-02,  2.3184e-01,
          2.9173e-01, -4.2566e-02,  2.3975e-01,  3.5194e-01, -1.9787e-02,
          1.7996e-01,  2.9437e-01, -2.7870e-03,  1.7775e-01,  2.4547e-01,
         -4.5525e-02,  1.8051e-01,  3.1298e-01, -3.7571e-02,  1.9058e-01,
          3.6134e-01, -1.7990e-02]])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x62 and 63x1024)

In [29]:
#saving model
torch.save(model,'model_f.pt')